In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
gloveFile = 'glove.42B.300d.txt'
with open(gloveFile, encoding="utf8" ) as f:
    #content = f.readlines()
    model = {}
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
print ("Done.",len(model)," words loaded!")

Done. 1917494  words loaded!


In [34]:
train_df = pd.read_pickle('train_df.pickle').sample(frac = 1)

In [63]:
train_df_t = (train_df[["text", "label"]])
mod_data = train_df_t.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.50 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 6116/12231 as positives and unlabeling the rest


In [64]:
mod_data['class_test'] = 0
mod_data.iloc[pos_sample,-1] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 0    29449
1     6116
Name: class_test, dtype: int64


In [65]:
mod_data

,text,label,class_test
10901,ActsisMMLJ CorrectlyObeyed God is a faggot and...,1,0
13070,This shit not funny or cool if a nigga ever sp...,1,1
31646,These bitches needs to be shot and killed This...,1,0
21455,Idk what Im watching but this bitch got a Rang...,0,0
67928,Shut up you little bitch Ill fuck you,1,0
...,...,...,...
29679,How about if u find out I blasted my sperm all...,0,0
14433,Please FB We must eliminate the criminal and h...,0,0
40218,just read the bible and stop trying to be prof...,0,0
30858,@DisneyStudios Westerners need to take of thei...,0,0


In [66]:
x_data = mod_data['text'].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [67]:
tokenizer = Tokenizer(num_words= 30000)
tokenizer.fit_on_texts(x_data)

In [68]:
embedding_matrix = np.zeros((30000, 300))
for word, index in tokenizer.word_index.items():
    if index > 30000 - 1:
        break
    else:
        if word in model.keys():
            embedding_matrix[index] = model[word]  
        else:
            embedding_matrix[index] = model['unk']  

In [69]:
X = tokenizer.texts_to_sequences(x_data)

In [70]:
X = pad_sequences(X, padding = 'pre', maxlen = 128)

In [71]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Embedding

In [72]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [45]:
# bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [46]:
# bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [73]:
model_glove = Sequential()
model_glove.add(Embedding(30000,300, input_length=128, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Bidirectional(LSTM(128)))
model_glove.add(Dense(1, activation='sigmoid'))
opt = Adam(learning_rate = 0.01)
model_glove.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4,restore_best_weights =True)

In [74]:

model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [75]:
# find the indices of the positive/labeled elements

positives = np.where(y_labeled == 1.)[0] 
print(positives)

[    1    13    25 ... 35547 35549 35550]


In [76]:
# hold_out_size = the *number* of positives/labeled samples 
# that we will use later to estimate P(s=1|y=1)
hold_out_ratio = 0.25
hold_out_size = int(np.ceil(len(positives) * hold_out_ratio))
np.random.shuffle(positives)

hold_out = positives[:hold_out_size] 
X_hold_out = X[hold_out]
X_hold_out[:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   146,
          110,   382,    17,    44,  2284,    16,    20,    17,   463,
          391, 10526,   839, 16481, 16482,    28,    22,   177,   141,
           44,  1802,  6620,   479,  1684,    82,   672, 29308,   510,
      

In [77]:
# the actual positive *elements* that we will keep aside
 
# remove the held out elements from X and y
X = np.delete(X, hold_out,0) 
y = np.delete(y_labeled, hold_out)

In [78]:
X

array([[    0,     0,     0, ...,    79,     4, 18557],
       [    0,     0,     0, ...,  1383,   179,   129],
       [    0,     0,     0, ...,     3,  3224,    50],
       ...,
       [    0,     0,     0, ...,   915,   438,  1474],
       [    0,     0,     0, ...,     9,    15,  7067],
       [    0,     0,     0, ...,     4,   132,  2044]])

In [79]:
X_train,X_val, y_train,y_val = train_test_split(X,y,test_size= 0.2, stratify=y)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2,restore_best_weights =True)
print(np.asarray(X_train)[:5])
model_glove.fit(np.asarray(list(X_train)),np.array(list(y_train)),
          validation_data =( np.array(list(X_val)),np.asarray(list(y_val))), epochs = 5, batch_size = 32,callbacks = [early_stopping])
#         # We then use the estimator for prediction of the positive held-out set 
# in order to estimate P(s=1|y=1)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0   106   405     1    60  1285    20  2987
   1177    37    45     2    35    28    32  1167]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0

In [80]:
hold_out_predictions = model_glove.predict(X_hold_out)
hold_out_predictions

array([[0.12067565],
       [0.06010506],
       [0.12735298],
       ...,
       [0.02753818],
       [0.22996056],
       [0.28060547]], dtype=float32)

In [81]:
#take the probability that it is 1
hold_out_predictions = hold_out_predictions[:,0]
# save the mean probability 
c = np.mean(hold_out_predictions)


def predict_PU_prob(X, estimator, prob_s1y1):
    predicted_s = estimator.predict_proba(X)
    predicted_s = predicted_s[:,1]
    return predicted_s / prob_s1y1

In [82]:
c

0.20701097

In [83]:
mod_data

,text,label,class_test
10901,ActsisMMLJ CorrectlyObeyed God is a faggot and...,1,0
13070,This shit not funny or cool if a nigga ever sp...,1,1
31646,These bitches needs to be shot and killed This...,1,0
21455,Idk what Im watching but this bitch got a Rang...,0,0
67928,Shut up you little bitch Ill fuck you,1,0
...,...,...,...
29679,How about if u find out I blasted my sperm all...,0,0
14433,Please FB We must eliminate the criminal and h...,0,0
40218,just read the bible and stop trying to be prof...,0,0
30858,@DisneyStudios Westerners need to take of thei...,0,0


In [84]:
test_ds = pd.read_pickle('test_df.pickle')

In [85]:
X_test = pad_sequences(tokenizer.texts_to_sequences(test_ds['text'].values),  padding = 'pre', maxlen = 128)
y_test = test_ds['label'].values

In [86]:
y_pred =  model_glove.predict(X_test)
y_pred

array([[0.17909795],
       [0.18174441],
       [0.3320802 ],
       ...,
       [0.02698931],
       [0.1323841 ],
       [0.03857805]], dtype=float32)

In [87]:
y_pred = y_pred/c

y_pred2 = [1 if i[0]>0.5 else 0 for i in y_pred]

In [88]:
from sklearn.metrics import classification_report

print(classification_report(test_ds['label'].values, y_pred2))

              precision    recall  f1-score   support

           0       0.80      0.67      0.73      2000
           1       0.72      0.83      0.77      2000

    accuracy                           0.75      4000
   macro avg       0.76      0.75      0.75      4000
weighted avg       0.76      0.75      0.75      4000



In [33]:
import os
 
pid = os.getpid()
!kill -9 $pid

'kill' is not recognized as an internal or external command,
operable program or batch file.
